# OCR-mLLM Pipeline

## 1. Run pytesseract

In [45]:
import os
from pathlib import Path
import pytesseract
from PIL import Image

# Get the user's path for the images folder assuming all images are stored here in .png format
source_dir = Path.cwd().parent.parent / "images"

# Get the user's path for the output folder, create one if it doesn't exist
target_dir.mkdir(parents=True, exist_ok=True)
target_dir = Path.cwd().parent.parent / "output"

# Add all filenames in images directory into the `filenames` array with the ENTIRE filepath
img_filepaths = []
for path in source_dir.iterdir():
  if path.is_file():
    img_filepaths.append(path)

# txt_filenames = []
# for filename in os.listdir(str(target_dir)):
#   if os.path.isfile(os.path.join(target_dir, filename)) :
#     txt_filenames.append(os.path.join(target_dir, filename))

In [46]:
for path in img_filepaths:
    file_name = target_dir / path.stem
    # file_name.touch()
    file_name = str(file_name) + ".txt"
    
    with open(file_name, 'w') as file:
        file.write(pytesseract.image_to_string(Image.open(str(path)))) # TODO: Change config as needed